<a href="https://colab.research.google.com/github/Rocks-n-Code/PythonCourse/blob/master/5%20-%20Lasio%20%26%20Striplog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<Back](https://colab.research.google.com/github/Rocks-n-Code/PythonCourse/blob/master/4%20-%20Geospatial%20Mapping.ipynb) [Next>](https://colab.research.google.com/github/Rocks-n-Code/PythonCourse/blob/master/6%20-%20Scraping%20Data.ipynb)

# 5 - Lasio & Striplog 
## by Thomas Martin

Adapted from Thomas's talk at Transform 2020 - [Video](https://youtu.be/oytSwhqvKbc) - [Repo](https://github.com/ThomasMGeo/Transform2020)

![Thomas Martin](https://github.com/Rocks-n-Code/PythonCourse/blob/master/img/TM130160.jpg?raw=true)

[Thomas Martin](https://twitter.com/ThomasM_geo) is a graduate student at [Colorado School of Mines](https://www.mines.edu/), in the [CoRE](https://core.mines.edu/) research group. He is working on core-log automated interepertation using machine learning. Feel free to reach out on [twitter](https://twitter.com/ThomasM_geo) or catch him on the software underground slack! 


There are whole classes devoted to basic python syntax, loops, functions, etc. This class is to just get you up and running with some key basics that can help with basic projects. Join us on the slack for some pointers!

## Importing and Using Packages

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Importing a few others that I use in (almost) every project. We can import them normally becuase Google pre-loaded them. Google does not pre-load everything that you need, sometimes you might need to install it.  ~*warning* ~ not every package can easily be imported in Colab. I have not figured out what makes some go and some not. But stats and plotting is usually pretty safe. Niche, one off, old, academic code, less so. 

In [ ]:
!pip install lasio # the ! before the pip is important, also this is a shell command, not a python one

Just installed the lasio package using pip. Lasio is a .las file imput and output reader. It's a great way to read in a .las file (well logs, not LiDAR) to use in python. After installing it, you still need to import it. The github for lasio is [here](https://github.com/kinverarity1/lasio)

In [ ]:
import lasio

There are hundreds of packages, not all of them work with Colab. We will install and import packages throughout the notebook.

# Well log input and output using lasio

lasio (.las input and output) is one of my all time favorite packages. It's updated often, and it works 99.5% of the time with .las files (well logs, not LiDAR data). The repo is [here](https://github.com/kinverarity1/lasio) on github. 

In [ ]:
las = lasio.read('https://raw.githubusercontent.com/ThomasMGeo/Transform2020/master/t20-intro/4900722147_722147B.las')

We just read in a las file! Let's do a quick quaility control, and make some plots.

In [ ]:
dir(las)

So there is a lot of stuff going on with the las object we just created. Which is awesome! But can be daunting to get into what you are looking for.

In [ ]:
las.curves

Wow, this well has a lot of curves! OK, we are going to make a data frame for the data in the .las file. This will behave really similar to the previous data frame

In [ ]:
data = las.df() # This time we named the dataframe data and not df
data.head()

Scroll around! If you want to get a handle on stats of your well curves, using the describe function on a dataframe is awesome. 

In [ ]:
data.to_csv('well_data.csv') # just saved it out!

Once this is in a data frame, you can export the curve data as a csv! This can be used in excel, spotfire, matlab or any other program you are more familiar with. This alone saves tons of time! While I prefer python, I know it's not for everyone.

In [ ]:
data.describe()

If you want different percentiles in df.describe, you can pass those values

In [ ]:
data.describe(percentiles=[0.1, 0.5, 0.9])

If you want to save out this table as a csv, you can do that in one line:

In [ ]:
data.describe().to_csv("petro-stats.csv")

In [ ]:
data.AHO10.min() # If you just want one of the stats

Let's use the quantile function to make some quick cutoffs (will be used in the future).

In [ ]:
lowGR = data.GR.quantile(.20)
lowGR

In [ ]:
highRES = data.AHO90.quantile(.95)
highRES

Feel free to make your own cutoffs here!

### Plotting up well log data

In [ ]:
# Cross plot comparing GR to Deep Resistivity
plt.figure(figsize=(5,4), dpi=100) # figure size and dpi you can set here
plt.scatter(data.GR.values, data.AHO90.values, color='blue', marker='.', alpha=0.3)
plt.yscale('log') #log scale for Y axis

plt.grid(True)
plt.xlabel('GR', size=16)
plt.ylabel('Deep Resistivity', size=16)

In [ ]:
# lets make a histogram of the GR Curve, with 20 bins, in green
plt.hist(data.GR.values, bins = 20, color='g', alpha = 0.5)
plt.xlabel('GR', size=16)
plt.ylabel('Count', size=16)

In [ ]:
# line plot of the Gamma curve
plt.figure(figsize=(4,8), dpi=100)
plt.plot(data.GR.values, data.index, color='g')
plt.ylabel('Depth in Feet', size=16)
plt.ylim(2000,0)
plt.grid(True)


Let's add a depth track for deep resistivity, a title and a few other additional widigits. Also we will use the well name for the title:

In [ ]:
las.header['Well'].WELL.value # we can pull the well name directly from the header

In [ ]:
# line plot of the Gamma curve
plt.figure(figsize=(6,7), dpi=100)

plt.suptitle(las.header['Well'].WELL.value, size =16) # overall title

plt.subplot(121) # if we are going to make two plots, matplotlib calls it a subplot
plt.plot(data.GR.values, data.index, color='g') # the actual plot!
plt.ylabel('Depth in Feet', size=16)
plt.xlabel('API', size=16)
plt.ylim(2000,0) # the limit is reversed to go deep down
plt.grid(True) # Turning the grid on

plt.subplot(122)
plt.plot(data.AHO10.values, data.index, color='grey')
plt.plot(data.AHO90.values, data.index, color='black')
plt.xscale('log')
plt.grid(True)
plt.xlim(0.4,4000)
plt.ylim(2000,0)
plt.xlabel('OhmM', size=16)
plt.tick_params(labelleft=False)  

plt.savefig('awesome-plot.pdf', dpi=200)

Let's add a pay flag, remember the cutoffs we calcualted above? Let's use those. Using pythonic nomenclature, we will set a true flag, only when both conditions are met.

In [ ]:
payMapper = (data.GR.values <= lowGR) & (data.AHO90.values >= highRES)
np.unique(payMapper)

If we want to see how many true and false statements there are:

In [ ]:
np.bincount(payMapper) # your specific numbers might be a bit different

So the GR values have to be below the cutoff, and resistivity have to be above the cutoff.

In [ ]:
# line plot of the Gamma curve
plt.figure(figsize=(5,8), dpi=100)

plt.subplot(131) # the subplot changed to allow for 3 columns
plt.plot(data.GR.values, data.index, color='g')
plt.ylabel('Depth in Feet', size=16)
plt.xlabel('API', size=16)
plt.ylim(2000,0)
plt.grid(True)

plt.subplot(132) # this is the new pay flag plot
plt.fill_between(payMapper, data.index, color='red') # used fill between, not plot
plt.ylim(2000,0)
plt.tick_params(labelleft=False)  
plt.xlabel('Pay Flag', size=16)
plt.grid(True)

plt.subplot(133)
plt.plot(data.AHO90.values, data.index, color='black')
plt.xscale('log')
plt.grid(True)
plt.xlim(0.4,4000)
plt.ylim(2000,0)
plt.xlabel('OhmM', size=16)
plt.tick_params(labelleft=False)  



Little heavy on the pay flag! But a good template for future work. Can be used for any sort of cutoff, or statistical analysis. Mess around with a plot below!

# Striplog

Striplog is an awesome package for  basic graphic logs, stratigraphic information, formations, and other geo stuff. The github is [here](https://github.com/agile-geoscience/striplog). This heavily borrows from [tutorials](https://github.com/agile-geoscience/striplog/tree/master/tutorial).

In [ ]:
!pip install striplog

In [ ]:
import striplog
striplog.__version__ #if this fails, just re run the above cells

### Lexicon

Striplog has a lot of geowords already preloaded. Think sand, shale, mudstone, salt, etc:

In [ ]:
from striplog import Lexicon
print(Lexicon.__doc__)

In [ ]:
lexicon = Lexicon.default()
lexicon #scroll around!

In [ ]:
lexicon.synonyms

These abbrevations are common for mudlogs, there is a great turtorial on the github if you are interested in that

In [ ]:
s = "grysh gn ss w/ sp gy sh"
lexicon.expand_abbreviations(s)

### Componet

In [ ]:
from striplog import Component

In [ ]:
print(Component.__doc__)

We define a new rock with a Python dict object:

In [ ]:
r = {'colour': 'grey',
     'grainsize': 'vf-f',
     'lithology': 'sand'}
rock = Component(r)
rock

You can now call these componets!

In [ ]:
rock['colour'] # who spelled this?! < Matt Hall/>

In [ ]:
rock.summary()

In [ ]:
rock.summary(fmt="My rock: {lithology} ({colour}, {grainsize!u})")



The formatting supports the usual s, r, and a:

    s: str
    r: repr
    a: ascii

Also some string functions:

    u: str.upper
    l: str.lower
    c: str.capitalize
    t: str.title

And some numerical ones, for arrays of numbers:

    + or ∑: np.sum
    m or µ: np.mean
    v: np.var
    d: np.std
    x: np.product



### Position

Positions define points in the earth, like a top, but with uncertainty. You can define:

    upper — the highest possible location
    middle — the most likely location
    lower — the lowest possible location
    units — the units of measurement
    x and y — the x and y location (these don't have uncertainty, sorry)
    meta — a Python dictionary containing anything you want

Positions don't have a 'way up'.

In [ ]:
from striplog import Position
print(Position.__doc__)

In [ ]:
params = {'upper': 95,
          'middle': 100,
          'lower': 110,
          'meta': {'kind': 'erosive', 'source': 'DOE'}
          }

p = Position(**params)
p

Even if you don't give a middle, you can always get z: the central, most likely position:

In [ ]:
params = {'upper': 75, 'lower': 85}
p = Position(**params)
p

In [ ]:
p.z

## Let's make a striplog!

I just introduced a bunch of boring things about dictionaries, etc. What makes this nice once you get it setup, is you can make some templates for common formations and lithotypes. We are going to do a simple one from this [github](https://github.com/ThomasMGeo/CSV2Striplog)

In [ ]:
from striplog import Lexicon, Decor, Component, Legend, Interval, Striplog


### Make a legend

Most of the stuff in the dicts you made were about display — so they are going to make Decor objects. A collection of Decors makes a Legend. A Legend determines how a striplog is displayed.

First I'll make the components, since those are easy. I'll move 'train' into there too, since it is to do with the rocks, not the display. If it seems weird having 'train' in the Component (which is really supposed to be about direct descriptions of the rock, but the idea is that it's always the same for all specimens of that rock so it does fit here) then you could put it in data instead.


In [ ]:
facies = {
    's': Component({'lithology': 'sandstone', 'train':'y'}),
    'i': Component({'lithology': 'interbedded', 'train':'y'}),
    'sh': Component({'lithology': 'shale', 'train':'y'}),
}

The next block of text could be less lines of code. The indenting is just a way to make it easier to read. Everyone has there own style of programming. We are just setting the decor of our future striplog plot. 

In [ ]:
sandstone = Decor({
    'component': facies['s'],
    'colour': 'yellow',
    'hatch': '.',
    'width': '3',
})

interbedded = Decor({
    'component': facies['i'],
    'colour': 'darkseagreen',
    'hatch': '--',
    'width': '2',
})

shale = Decor({
    'component': facies['sh'],
    'colour': 'darkgray',
    'hatch': '-',
    'width': '1',
})


In [ ]:
legend = Legend([sandstone, interbedded, shale])
legend

In [ ]:
#Read in file to Colab instance
loc = 'https://raw.githubusercontent.com/ThomasMGeo/Transform2020/master/t20-intro/t20-lith.csv'
lith = pd.read_csv(loc)
lith.to_csv('t20-lith.csv',
            index=False)

In [ ]:
strip = Striplog.from_csv('t20-lith.csv')
strip[0]


##Deal with lithology¶

The lithology has been turned into a component, but it's using the abbreviation... I can't figure out an elegant way to deal with this so, for now, we'll just loop over the striplog and fix it. We read the data item's lithology ('s' in the top layer), then look up the correct lithology name in our abbreviation dictionary, then add the new component in the proper place. Finally, we delete the data we had.


In [ ]:
strip[0].data

In [ ]:
for s in strip:
    lith = s.data['lithology']
    s.components = [facies[lith]]
    s.data = {}
# Run once, not twice

In [ ]:
strip[0]

In [ ]:
strip.plot(legend)

Just plotted a simple strip log from a CSV! You can make striplogs 100 different ways, and I highly reccomend the turtorials on the github for more exploration. Striplog is a fun package to dig into (pun intended) as it forces you to think about how this data is stored, managed, and used.

[<Back](https://colab.research.google.com/github/Rocks-n-Code/PythonCourse/blob/master/4%20-%20Geospatial%20Mapping.ipynb) [Next>](https://colab.research.google.com/github/Rocks-n-Code/PythonCourse/blob/master/6%20-%20Scraping%20Data.ipynb)